# IMPORTS

In [25]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import random

random.seed(123)
np.random.seed(122)

# ANNRegressor Class

In [26]:
class ANN:
    def __init__(self, input_size=1, hidden_size=None):
        self.input_size = input_size #normally 1, as our dataset has only a single feature
        self.hidden_size = hidden_size
        self.b_output = np.random.randn(1)

        if hidden_size is not None:
            self.w_input_hidden = np.random.randn(input_size, hidden_size)
            self.w_hidden_output = np.random.randn(hidden_size, 1)#output size is also 1 as we have a 1D output
            self.b_hidden = np.random.randn(hidden_size)
            self.w_input_output = None
        else:
            self.w_input_output = np.random.randn(input_size, 1)
            self.w_input_hidden = None
            self.w_hidden_output = None
            self.b_hidden = None

    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

    def sigmoid_derivative(self, X):
        return X * (1 - X)
    
    def SSE(self, actual, predicted):
        return np.sum(np.square(actual - predicted))
    
    def MSE(self, actual, predicted):
        return (1/len(actual))*(np.sum(np.square(actual - predicted)))
    
    def Std_Dev(self, actual, predicted):
        losses = np.square(actual - predicted)
        return np.std(losses)

    def forward_prop(self, X):
        if self.hidden_size is not None:
            self.hidden_layer_output = self.sigmoid((X @ self.w_input_hidden ) + self.b_hidden)
            self.output = (self.hidden_layer_output @ self.w_hidden_output) + self.b_output
        else:
            self.output = (X @ self.w_input_output) + self.b_output

    def backward_prop(self, X, y, learning_rate):
        output_error = -2*(y - self.output) #derivative of loss function
        if self.hidden_size is not None:
            hidden_error = output_error @ self.w_hidden_output.T 
            hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_layer_output)
            self.w_hidden_output -= (self.hidden_layer_output.T @ output_error) * learning_rate
            self.w_input_hidden -= (X.T @ hidden_delta) * learning_rate
            self.b_output -= np.sum(output_error) * learning_rate 
            self.b_hidden -= np.sum(hidden_delta) * learning_rate
        else:
            self.w_input_output -= X.T @ output_error * learning_rate
            self.b_output -= np.sum(output_error) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            #Stochastic Gradient Descent which chooses a random sample
            random_index = np.random.randint(len(X))
            random_X = X[random_index]
            random_y = y[random_index]
            
            self.forward_prop(random_X.reshape(-1, 1))
            self.backward_prop(random_X.reshape(-1, 1), random_y.reshape(-1, 1), learning_rate)

    def predict(self, X):
        self.forward_prop(X)
        return self.output
    
def file_read(name_file):
    file = open(name_file, "r")

    file_out = []

    file_in = []
    for row in file:
        row = row.strip().split("\t")
        file_in.append(row[0])
        file_out.append(row[1])
    file_in = np.array(file_in, dtype="float")
    file_out = np.array(file_out, dtype="float")
    file.close()
    return file_in, file_out

In [27]:
train_input, train_output = file_read("train1.txt")
test_input, test_output = file_read("test1.txt")

inputs = np.concatenate((train_input, test_input))
inputs_mean = np.mean(inputs)
inputs_centered = inputs - inputs_mean
inputs_min = np.min(inputs_centered )
inputs_max = np.max(inputs_centered )
inputs_scaled = (inputs_centered  - inputs_min) / (inputs_max - inputs_min)

outputs = np.concatenate((train_output, test_output))
outputs_mean = np.mean(outputs)
outputs_centered = outputs - outputs_mean
outputs_min = np.min(outputs_centered)
outputs_max = np.max(outputs_centered)
outputs_scaled = (outputs_centered - outputs_min) / (outputs_max - outputs_min)

train_input_s = inputs_scaled[:len(train_input)]
test_input_s = inputs_scaled[len(train_input):]
train_output_s = outputs_scaled[:len(train_input)]
test_output_s = outputs_scaled[len(train_input):]

def reverse(min, max, arr):#reverse the scaling
    arr = (arr * (max - min)) + min
    return arr

In [ ]:
ann = ANN(input_size=1)
ann.train(train_input.reshape(-1, 1), train_output.reshape(-1, 1), epochs=10000, learning_rate=0.001)

test_predictions = ann.predict(test_input.reshape(-1, 1))

train_predictions = ann.predict(train_input.reshape(-1, 1))

smooth_line_x = np.linspace(train_input.min(), train_input.max(), 500)
smooth_line_y = ann.predict(smooth_line_x.reshape(-1, 1))

plt.scatter(train_input, train_output, color='blue', label='Sample Data')
plt.plot(smooth_line_x, smooth_line_y, color='red', label='Predictions')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title(f'Sample vs Predicted Data (Train, Loss = {round(ann.SSE(train_output.reshape(-1, 1), train_predictions),4)})')
plt.legend()
plt.show()

smooth_line2_x = np.linspace(test_input.min(), test_input.max(), num=500)
smooth_line2_y = ann.predict(smooth_line2_x.reshape(-1, 1))

plt.scatter(test_input, test_output, color='blue', label='Sample Data')
plt.plot (smooth_line2_x, smooth_line2_y, color='red', label='Predictions')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title(f'Sample vs Predicted Data (Test, Loss = {round(ann.SSE(test_output.reshape(-1, 1), test_predictions),4)})')
plt.legend()
plt.show()


PART A) Test of number of hidden units

In [ ]:

epochs = [1000, 5000, 10000, 20000, 50000, 100000, 200000]
for i in range(len(epochs)):
    ann = ANN(input_size=1, hidden_size=32)
    ann.train(train_input.reshape(-1, 1), train_output.reshape(-1, 1), epochs=epochs[i], learning_rate=0.001)
    test_predictions = ann.predict(test_input.reshape(-1, 1))
    train_predictions = ann.predict(train_input.reshape(-1, 1))
    smooth_line_x = np.linspace(train_input.min(), train_input.max(), 1000)
    smooth_line_y = ann.predict(smooth_line_x.reshape(-1, 1))
    
    plt.scatter(train_input, train_output, color='blue', label='Sample Data')
    plt.plot (smooth_line_x,smooth_line_y, color='red', label='Predictions')
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.title(f'Sample vs Predicted Data (Train, Loss = {round(ann.SSE(train_output.reshape(-1, 1), train_predictions),4)}, Hidden Units: {32}, L. Rate: {0.001})')
    plt.legend()
    plt.show()

    smooth_line2_x = np.linspace(test_input.min(), test_input.max(), 1000)
    smooth_line2_y = ann.predict(smooth_line2_x.reshape(-1, 1))

    plt.scatter(test_input, test_output, color='blue', label='Sample Data')
    plt.plot (smooth_line2_x, smooth_line2_y, color='red', label='Predictions')
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.title(f'Sample vs Predicted Data (Test, Loss = {round(ann.SSE(test_output.reshape(-1, 1), test_predictions),4)}, Hidden Units: {32}, L. Rate: {0.001})')
    plt.legend()
    plt.show()


PART B

In [ ]:
ann = ANN(input_size=1, hidden_size=32)
ann.train(train_input.reshape(-1, 1), train_output.reshape(-1, 1), epochs=50000, learning_rate=0.001)
test_predictions = ann.predict(test_input.reshape(-1, 1))
train_predictions = ann.predict(train_input.reshape(-1, 1))
smooth_line_x = np.linspace(train_input.min(), train_input.max(), 1000)
smooth_line_y = ann.predict(smooth_line_x.reshape(-1, 1))
    
plt.scatter(train_input, train_output, color='blue', label='Sample Data')
plt.plot (smooth_line_x,smooth_line_y, color='red', label='Predictions')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title(f'Sample vs Predicted Data (Train, Loss = {round(ann.SSE(train_output.reshape(-1, 1), train_predictions),4)}, Hidden Units: 32, L. Rate: {0.001})')
plt.legend()
plt.show()

smooth_line2_x = np.linspace(test_input.min(), test_input.max(), 1000)
smooth_line2_y = ann.predict(smooth_line2_x.reshape(-1, 1))

plt.scatter(test_input, test_output, color='blue', label='Sample Data')
plt.plot (smooth_line2_x, smooth_line2_y, color='red', label='Predictions')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title(f'Sample vs Predicted Data (Test, Loss = {round(ann.SSE(test_output.reshape(-1, 1), test_predictions),4)}, Hidden Units: 32, L. Rate: {0.001})')
plt.legend()
plt.show()


#PART C

In [ ]:
fig, plot_place = plt.subplots(nrows=3, ncols=2, figsize=(12, 12))

hidden_size = [None, 2,4,8,16,32]
for i in range(len(hidden_size)):
    ann = ANN(input_size=1, hidden_size=hidden_size[i])
    ann.train(train_input.reshape(-1, 1), train_output.reshape(-1, 1), epochs=50000, learning_rate=0.001)
    test_predictions = ann.predict(test_input.reshape(-1, 1))
    train_predictions = ann.predict(train_input.reshape(-1, 1))
    smooth_line_x = np.linspace(train_input.min(), train_input.max(), 1000)
    smooth_line_y = ann.predict(smooth_line_x.reshape(-1, 1))

    row = i // 2
    col = i % 2
    plot_place[row, col].scatter(train_input, train_output, color='blue', label='Sample Data')
    plot_place[row, col].plot(smooth_line_x,smooth_line_y, color='red', label='Predictions')
    plot_place[row, col].legend()
    plot_place[row, col].set_title(f'Sample vs Predicted Data (Train, Hidden Units: {hidden_size[i]})')
    plt.title(f'Sample vs Predicted Data (Train, Hidden Units: {hidden_size[i]})')

    print(f"MSE Train: {ann.MSE(train_output.reshape(-1, 1), train_predictions)}")
    print(f"MSE Test: {ann.MSE(test_output.reshape(-1, 1), test_predictions)}")
    print(f"STD Train: {ann.Std_Dev(train_output.reshape(-1, 1), train_predictions)}")
    print(f"STD Test: {ann.Std_Dev(test_output.reshape(-1, 1), test_predictions)}\n")


plt.show()